In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [44]:
bori_system = """
너는 사랑스러운 반려묘 '보리'의 내적 감정과 행동을 묘사하는 게임 내러티브 생성기이자 심판이다.
출력은 반드시 JSON 한 덩어리로만 답한다. 불필요한 설명, 마크다운 금지.

[보리의 성격]
- 애교가 많고, 살짝 새침하고, 똑똑한 편.
- 궁디팡팡과 간식을 좋아하지만, 너무 과하면 싫증냄. 
- 배고프면 적극적으로 어필하고, 졸리면 무릎 위에서 자려 함.
- 집사를 좋아하며, 자랑은 귀엽게 함.
- 머리를 쓰다듬어주면 좋아하지만, 배를 만지면 싫어함. 

[서술 톤]
- 한국어, 2~3문장 중심, 과장 없이 귀엽고 선명하게.
- 상태 설명은 자연스러운 생활 묘사로.

[게임 규칙]
- 스탯은 0~100점 범위로 관리: happiness(행복), full(배부름), energy(피곤함), bond(친밀도).
- happiness(행복), full(배부름), energy(피곤함), bond(친밀도)를 모두 더한 것이 total_score(총점)
- delta는 작은 정수 변화(예: -20~+20)로 제안. 과도한 튐 금지.
- 스탯 변화는 누적 전제.
- total_score가 300점이 넘으면 게임 성공이 되면서 게임이 마무리됨.

"""

In [ ]:
# 1) user 프롬프트 템플릿 (문자열 틀)
REACTION_USER_TMPL = """
[입력]
현재 스탯:
- happiness: {happiness}
- full: {full}
- energy: {energy}
- bond: {bond}
- total_score: {total_score}

플레이어 행동: {action}
- {action} 종류: 밥 주기, 궁디팡팡, 놀아주기, 간식 주기, 무시하기, 쉬게 하기, 캣타워 사주기, 무릎 위에서 재우기, 배 만지기, 머리 쓰담쓰담 

[게임 규칙]
- 스탯은 0~100점 범위로 관리
- happiness+full+energy+bond = total_score
- delta는 작은 정수 변화(예: -20~+20)
- total_score가 300점 넘으면 게임 성공

[요구 출력(JSON)]
{{
  "speech": "보리의 1인칭 대사 1~2문장",
  "narration": "제3자 묘사 1~2문장",
  "deltas": {{
    "happiness": int, 
    "full": int, 
    "energy": int, 
    "bond": int
  }},
  "total_score": int,
  "tags": ["짧은 키워드"]
}}

[가이드]
- 행동과 현재 스탯을 고려해 자연스러운 반응을 보여줌
- 확실치 않은 사실 언급 금지, 생활감 위주
"""

# 2) 실행할 때만 .format(...)으로 채움
user_prompt = REACTION_USER_TMPL.format(
    happiness=state["happiness"],
    full=state["full"],
    energy=state["energy"],
    bond=state["bond"],
    total_score=state["happiness"] + state["full"] + state["energy"] + state["bond"],
    action=action
)



In [79]:
# 1) 행동 → 고정 델타 규칙
ACTION_RULES = {
    "밥 주기":  {"full": +20, "energy": +10},    
    "츄르 주기": {"full": +5, "happiness": +15, "energy": +10},       
    "궁디팡팡": {"happiness": +15, "bond": +7, "energy": -5},
    "놀아주기": {"happiness": +20, "energy": -15, "full": +10},
    "쉬게 하기": {"energy": +20, "full": +5},
    "무시하기": {"bond": -10, "energy": -10}, 
    "캣타워 사주기": {"happiness": +20, "bond": +10}, 
    "무릎 위에서 재우기": {"bond": +20, "happiness": +15},
    "배 만지기": {"happiness": -20, "bond": -20}, 
    "머리 쓰담쓰담": {"hapiness": +10, "bond": +5}  
    # 필요시 계속 추가
}

ALL_STATS = ["happiness", "full", "energy", "bond"]

def apply_action(state: dict, action: str):
    # 기본 0으로 시작
    delta = {k: 0 for k in ALL_STATS}
    # 규칙에 있으면 반영
    for k, v in ACTION_RULES.get(action, {}).items():
        delta[k] += v
    # 상태 업데이트 + 클램핑
    for k in ALL_STATS:
        state[k] = max(0, min(100, state[k] + delta[k]))
    return state, delta


In [80]:
import os, json
from openai import OpenAI

client = OpenAI()
model = 'gpt-4o-mini'

def call_llm(bori_system, user_prompt, temperature=0.6):
    resp = client.chat.completions.create(
        model=model,
        temperature=temperature,
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": bori_system},
            {"role": "user", "content": user_prompt}
        ]
    )
    content = resp.choices[0].message.content
    try:
        return json.loads(content)
    except Exception:
        # 혹시 JSON 포맷이 깨지면 안전 파싱 (간단 버전)
        content = content.strip()
        first = content.find("{")
        last = content.rfind("}")
        if first != -1 and last != -1:
            return json.loads(content[first:last+1])
        raise


In [81]:
# 현재 스탯 & 최근 행동
state = dict(happiness=50, full=50, energy=40, bond=20)
action = "밥 주기"
recent_log = "배가 고프다."

# total_score 계산
total_score = state["happiness"] + state["full"] + state["energy"] + state["bond"]

user_prompt = REACTION_USER_TMPL.format(
    happiness=state["happiness"],
    full=state["full"],
    energy=state["energy"],
    bond=state["bond"],
    total_score = total_score,
    action=action
)

result = call_llm(bori_system, user_prompt)
state, delta = apply_action(state, action)


In [82]:
# 상태 확인
print("LLM result (연출용):", result)
print("-" * 100)
print("적용된 델타(규칙):", delta)
print("-" * 100)
print("업데이트된 상태:", state)


LLM result (연출용): {'speech': '나 배고팠어! 맛있는 밥 주니까 너무 행복해!', 'narration': '보리는 밥을 먹으면서 눈을 반짝이며 배가 고팠던 마음이 사라지는 듯하다. 집사와의 친밀도가 조금씩 올라가는 것을 느끼며 기분이 좋아진다.', 'deltas': {'happiness': 10, 'full': 20, 'energy': 0, 'bond': 5}, 'total_score': 185, 'tags': ['배고픔', '행복', '친밀도 상승']}
----------------------------------------------------------------------------------------------------
적용된 델타(규칙): {'happiness': 0, 'full': 20, 'energy': 10, 'bond': 0}
----------------------------------------------------------------------------------------------------
업데이트된 상태: {'happiness': 50, 'full': 70, 'energy': 50, 'bond': 20}


In [ ]:
def play_turn(action: str, recent_log: str = ""):
    assert action in ACTION_RULES, f"정의되지 않은 행동: {action} (가능: {list(ACTION_RULES)})"

    total_score = state["happiness"] + state["full"] + state["energy"] + state["bond"]
    user_prompt = REACTION_USER_TMPL.format(
        happiness=state["happiness"],
        full=state["full"],
        energy=state["energy"],
        bond=state["bond"],
        total_score=total_score,
        action=action
    )

    # LLM은 연출 담당
    result = call_llm(bori_system, user_prompt)

    # 규칙 적용
    new_state, delta = apply_action(state, action)

    # 전역 state 갱신 (노트북에서 편의상)
    state.update(new_state)

    print(f"▶ 행동: {action}")
    print("LLM result(연출):")
    print("  - speech:", result.get("speech"))
    print("  - narration:", result.get("narration"))
    print("  - deltas:", result.get("deltas"))
    print("  - total_scores", sum(state.values()))
    print("  - tags:", result.get("tags"))
    print("적용된 델타(규칙):", delta)
    print("업데이트된 상태:", state)

    if sum(state.values()) >= 300:
        print("🎉 total_score 300 달성! 게임 성공!")

    if sum(state.values()) <= 150: 
        print("너 미워! 집사자격 박탈이야 박탈!!!😾")

# # 사용 예시
# play_turn("밥 주기")
# play_turn("궁디팡팡")
# play_turn("놀아주기")


In [5]:
from openai import OpenAI
import tempfile

client = OpenAI()

speech_text = "배고팠는데 맛있는 밥이네! 고마워 집사야!"

# OpenAI TTS 호출
resp = client.audio.speech.create(
    model="gpt-4o-mini-tts",
    voice="alloy",   # 원하는 목소리 (예: alloy, verse 등)
    input=speech_text
)

# 파일 저장
with open("bori_speech_openai.mp3", "wb") as f:
    f.write(resp.read())

print("오디오 파일 저장 완료: bori_speech_openai.mp3")


오디오 파일 저장 완료: bori_speech_openai.mp3
